In [1]:
import math as m
import numpy as np
import random as r
import matplotlib.pyplot as plt
import sys

In [2]:
import torch
from torch import nn
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR

In [3]:
from nflows.flows.base import Flow
from nflows.distributions.uniform import BoxUniform
from nflows.transforms.base import CompositeTransform
from nflows.transforms.autoregressive import MaskedPiecewiseRationalQuadraticAutoregressiveTransform
from nflows.transforms.autoregressive import MaskedPiecewiseQuadraticAutoregressiveTransform
from nflows.transforms.permutations import ReversePermutation
from nflows.transforms.permutations import RandomPermutation
from nflows.transforms.splines.rational_quadratic import rational_quadratic_spline
from torch.utils.tensorboard import SummaryWriter

In [4]:
import subprocess
import time
import os
from copy import deepcopy
import math as m
import gc

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
# Tensorboard writer
writer = SummaryWriter()

In [7]:
device = torch.device("cuda:0")
#device = torch.device("cpu")

In [8]:
# neg run 14
n_RQS_knots = 10   # Number of knots in RQS transform
n_made_layers = 3  # Number of layers in every made network
n_made_units = 500 # Number of units in every layer of the made network
n_flow_layers = 12 # Number of layers in the flow

batch_size = 1024
n_epochs = 800
adam_lr = 0.001   # Learning rate for the ADAM optimizer (default: 0.001)

In [9]:
n_testing = int(3.6820486488184496*1e5)

In [10]:
training_samples = torch.tensor(np.genfromtxt("neg_training_samples.csv",
                    delimiter=','), dtype=torch.float32, device=device)
testing_samples = torch.tensor(np.genfromtxt("neg_testing_samples.csv",
                    delimiter=',')[:n_testing], dtype=torch.float32, device=device)

In [11]:
training_weights = np.genfromtxt("neg_training_weights.csv", delimiter=',')
testing_weights = np.genfromtxt("neg_testing_weights.csv", delimiter=',')[:n_testing]

In [12]:
training_weights = torch.tensor(np.sign(training_weights), dtype=torch.float32, device=device)
testing_weights = torch.tensor(np.sign(testing_weights), dtype=torch.float32, device=device)

In [13]:
event_dim = training_samples.shape[1] # Dimensionality of data

In [14]:
base_dist = BoxUniform(torch.zeros(event_dim), torch.ones(event_dim))

transforms = []
for _ in range(n_flow_layers):
    transforms.append(RandomPermutation(features=event_dim))
    transforms.append(MaskedPiecewiseRationalQuadraticAutoregressiveTransform(
        features=event_dim, 
        hidden_features=n_made_units,
        num_bins=n_RQS_knots,
        num_blocks=n_made_layers-1,
        tails="constrained",
        use_residual_blocks=False
    ))
transform = CompositeTransform(transforms)

flow = Flow(transform, base_dist).to(device)
optimizer = optim.Adam(flow.parameters(), lr=adam_lr)

scheduler = MultiStepLR(optimizer, milestones=[350, 425, 500, 575, 650, 725, 800], gamma=0.5)

In [ ]:
data_size = training_samples.shape[0]
n_batches = m.ceil(data_size/batch_size)

data_size_validation = testing_samples.shape[0]
n_batches_validate = m.ceil(data_size_validation/batch_size)

best_loss = np.inf
for epoch in range(n_epochs):
    scheduler.step()
    
    permutation = torch.randperm(data_size, device=device)    

    # Loop over batches
    cum_loss = 0
    for batch in range(n_batches):
        # Set up the batch
        batch_begin = batch*batch_size
        batch_end   = min( (batch+1)*batch_size, data_size-1 )
        indices = permutation[batch_begin:batch_end]
        samples_batch = training_samples[indices]
        weights_batch = training_weights[indices]
        
        # Take a step
        optimizer.zero_grad()
        loss = -(flow.log_prob(inputs=samples_batch)*weights_batch).mean()
        loss.backward()
        optimizer.step()

        # Compute cumulative loss
        cum_loss = (cum_loss*batch + loss.item())/(batch+1)

        if batch%25 == 0:
            print("epoch = ", epoch, "batch = ",batch+1, "/", n_batches, "loss = ", cum_loss)
    
    # Compute validation loss
    validation_loss = 0
    for batch in range(n_batches_validate):
        batch_begin = batch*batch_size
        batch_end = min( (batch+1)*batch_size, data_size_validation-1 )
        samples_batch = testing_samples[batch_begin:batch_end]
        weights_batch = testing_weights[batch_begin:batch_end]
    
        with torch.no_grad():
            validation_loss = (validation_loss*batch - (flow.log_prob(samples_batch)*weights_batch).mean())/(batch+1)
    
    print("Validation loss = ", validation_loss)
    
    writer.add_scalar("Loss_train", cum_loss, epoch)
    writer.add_scalar("Loss_test", validation_loss, epoch)
    
    if validation_loss < best_loss:
        torch.save(flow, "flow_model.pt")
        best_loss = validation_loss

/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


epoch =  0 batch =  1 / 3596 loss =  6.753967761993408
epoch =  0 batch =  26 / 3596 loss =  -6.534637190544834
epoch =  0 batch =  51 / 3596 loss =  -10.456426861604639
epoch =  0 batch =  76 / 3596 loss =  -12.033402792962367
epoch =  0 batch =  101 / 3596 loss =  -13.069429122759859
epoch =  0 batch =  126 / 3596 loss =  -13.73941470626446
epoch =  0 batch =  151 / 3596 loss =  -14.215411261340838
epoch =  0 batch =  176 / 3596 loss =  -14.542447311740199
epoch =  0 batch =  201 / 3596 loss =  -14.78892212433379
epoch =  0 batch =  226 / 3596 loss =  -15.01673978997345
epoch =  0 batch =  251 / 3596 loss =  -15.225528534264322
epoch =  0 batch =  276 / 3596 loss =  -15.443562203194892
epoch =  0 batch =  301 / 3596 loss =  -15.60108390090325
epoch =  0 batch =  326 / 3596 loss =  -15.740978670689318
epoch =  0 batch =  351 / 3596 loss =  -15.865404593352324
epoch =  0 batch =  376 / 3596 loss =  -15.984581216585209
epoch =  0 batch =  401 / 3596 loss =  -16.077922578213613
epoch =  